In [ ]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [ ]:
import ee
import geemap
Map = geemap.Map()
Map

In [ ]:
roi = ee.FeatureCollection('users/311605001111/YangtzeCity/wuhan')
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,9)

In [ ]:
js_snippet = """



"""

In [ ]:
geemap.js_snippet_to_py(js_snippet)

In [ ]:
image = ee.Image('USDA/NAIP/DOQQ/m_4307663_se_18_1_20130619')

Map.setCenter(-76.13, 43.03,16)
Map.addLayer(image, {'max': 255}, 'Input image')

nir = image.select('N')
square = ee.Kernel.square(**{'radius': 3})
entropy = nir.entropy(square)

Map.addLayer(entropy,{'min': 1, 'max': 5, 'palette': ['0000CC', 'CC0000']}, 'entropy')

In [ ]:
histogram = entropy.select('N').reduceRegion(**{
    'reducer': ee.Reducer.histogram(100),
    'geometry': image.geometry(), 
    'scale': 100,
    'bestEffort': True
})
print(histogram.getInfo())

In [ ]:
# 图像融合
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_121038_20160115')
Map.addLayer(image,{'bands': ['B4', 'B3', 'B2'],'min':0,'max':3000}, 'RGB image')
hsv = image.select(['B4', 'B3', 'B2']).rgbToHsv()

combined = ee.Image.cat([hsv.select('hue'), hsv.select('saturation'), image_toa.select('B8')])

sharpened = ee.Image(combined).hsvToRgb()
Map.centerObject(image,8)
Map.addLayer(sharpened,'','pan-sharpened')

In [ ]:
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_015030_20140307').select("B8")
Map.setCenter(-76.1467, 43.0458,12)
Map.addLayer(image, '', 'input image')

canny = ee.Algorithms.CannyEdgeDetector(**{'image': image, 'threshold': 200, 'sigma': 1 })
Map.addLayer(canny, {}, 'canny')

hough = ee.Algorithms.HoughTransform(canny, 256, 7000, 80)
Map.addLayer(hough, {}, 'hough')